This notebook experiments with building out a fully customized TerriaJS catalog from a search result of ScienceBase Items, transforming sbJSON into the json catalog format used by TerriaJS. Resulting catalog JSON files are output to the local directory, deployed to GitHub, and then callable into an out of the box TerriaJS mapper.

In [1]:
import requests,json
from IPython.display import display

In [6]:
# Get what we want to work with using a ScienceBase /items/ query
sbURL = "https://www.sciencebase.gov/catalog/items?parentId=56bba648e4b08d617f657960&&filter0=browseType%3DArcGIS+REST+Map+Service&format=json&fields=title,body,distributionLinks&loadInstances=true"
sbJSON = requests.get(sbURL).json()



In [11]:
theCatalog = {}
theCatalog["corsDomains"] = ["sciencebase.gov","corsproxy.com"]
theCatalog["baseMapName"] = "Bing Maps Roads"
theCatalog["initialViewerMode"] = "2d"
theCatalog["services"] = []

theCatalog["homeCamera"] = {}
theCatalog["homeCamera"]["west"] = -175
theCatalog["homeCamera"]["east"] = -66
theCatalog["homeCamera"]["north"] = 72
theCatalog["homeCamera"]["south"] = 18

theCatalog["catalog"] = []

padusGroup = {}
padusGroup["name"] = "Protected Areas Database of the U.S."
padusGroup["type"] = "group"
padusGroup["isOpen"] = "true"

padusGroup["items"] = []

for idx, item in enumerate(sbJSON["items"]):
    thisItem = {}
    thisItem["id"] = item["id"]
    thisItem["name"] = item["title"]
    thisItem["description"] = item["body"]
    serviceCount = 0

    if idx < 4:
        thisItem["layers"] = "0"
        thisItem["isEnabled"] = "true"

    for link in item["distributionLinks"]:
        if link["title"] == "ArcGIS REST Service":
            serviceCount = serviceCount + 1
            if serviceCount > 1:
                break
            thisItem["type"] = "esri-mapServer"
            thisItem["url"] = link["uri"]
    padusGroup["items"].append(thisItem.copy())

theCatalog["catalog"].append(nfhpGroup.copy())

display (theCatalog)

f = open('catalog.json', 'wt', encoding='utf-8')
f.write(json.dumps(theCatalog).replace(': "true"', ': true'))

{'baseMapName': 'Bing Maps Roads',
 'catalog': [{'isOpen': 'true',
   'items': [{'description': 'General land manager description (e.g. Federal, Tribal, State, Private) standardized for the US. See PAD-US Data Standard for "Agency Name" to "Agency Type" crosswalk or look up table included in data download, for full domain descriptions. Use "Manager Type" rather than "Owner Type" for the most complete depiction of federal lands as overlapping designations (i.e. "Designation") and ownership related data gaps (i.e. "Unknown") occur in the federal theme.',
     'id': '56bba5b8e4b08d617f65795a',
     'isEnabled': 'true',
     'layers': '0',
     'name': 'PADUS by Manager Type',
     'type': 'esri-mapServer',
     'url': 'https://gis1.usgs.gov/arcgis/rest/services/PADUS1_4/General_Agency_Level/MapServer'},
    {'description': 'Level of public access permitted. Open requires no special requirements for public access to the property (may include regular hours available). Restricted requires a 

6096